In [ ]:
!pip install -U scikit-fuzzy


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 994.0/994.0 kB 13.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-fuzzy: filename=scikit_fuzzy-0.4.2-py3-none-any.whl size=894077 sha256=ecfa73b2da56544c2128f4e163c5d7eeccca82a5192e894ceb0ecedde849b308
  Stored in directory: /root/.cache/pip/wheels/4f/86/1b/dfd97134a2c8313e519bcebd95d3fedc7be7944db022094bc8
Successfully built scikit-fuzzy


In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

user_query_list = pd.read_csv('/content/drive/MyDrive/ddbss.csv', index_col='sites')
user_query_list = user_query_list.fillna('n')
user_query_list = user_query_list.sort_values(by=['sites'])
print(user_query_list)
data = user_query_list.to_numpy()
num_of_attributes = data[0].shape[0]
num_of_sites = data.shape[0]
for i in range(num_of_sites):
    for j in range(0, num_of_attributes):
        data[i][j] = 0 if data[i][j] == 'n' else 1
saum = np.array(data, dtype=np.int32)


aaum = np.dot(saum.T, saum)

from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2)
distances = kmeans.fit_transform(saum)
attr_fragment = np.zeros(len(distances), dtype=np.int32)
for i in range(len(distances)):
    attr_fragment[i] = np.argmin(distances[i]) + 1
distance_frag = np.column_stack((distances, attr_fragment))

df = pd.DataFrame(distance_frag)

df1 = df.iloc[:, 0:-1]
df1.columns = ["F1", "F2"]

fragment_allocation=np.zeros((saum.shape[0],2),dtype=np.int32)
for i in range(len(saum)):
    total_access = 0
    for j in range(len(saum[0])):
        if saum[i][j] == 1 and distance_frag[j][2] == 1:
            total_access += saum[i][j]
    fragment_allocation[i][0] = total_access

for i in range(len(saum)):
    total_access = 0
    for j in range(len(saum[0])):
        if saum[i][j] == 1 and distance_frag[j][2] == 2:
            total_access += saum[i][j]
    fragment_allocation[i][1] = total_access

frag_all = pd.DataFrame(fragment_allocation)
frag_all.columns = ['AttrFromFragmentOne', 'AttrFromFragmentTwo']
frag_all.index = user_query_list.index

# Print the result
print(frag_all)

        A1 A2 A3   A4 A5 A6 A7 A8   A9 A10
sites                                     
1        c  n  n    n  n  n  n  c    n   n
2        r  n  r    n  n  n  n  n    n   r
3        n  r  n    n  n  r  n  n    n   n
4        c  n  n    n  n  n  c  c    n   n
5        n  n  n  r,r  r  n  r  r    r   n
6        n  n  n    r  r  n  r  n    n   n
7        n  n  r    n  n  r  n  r    n   r
8        d  d  d    d  d  d  d  d  d,r   d
9      d,r  d  d    d  d  d  d  d    d   d
10       c  c  n    n  n  n  n  n    n   n
       AttrFromFragmentOne  AttrFromFragmentTwo
sites                                          
1                        1                    1
2                        0                    3
3                        1                    1
4                        1                    2
5                        3                    2
6                        1                    2
7                        2                    2
8                        4                    6
9   

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

user_query_list = pd.read_csv('/content/drive/MyDrive/ddbss.csv', index_col='sites')
user_query_list = user_query_list.fillna('n')
user_query_list = user_query_list.sort_values(by=['sites'])
print(user_query_list)
data = user_query_list.to_numpy()
num_of_attributes = data[0].shape[0]
num_of_sites = data.shape[0]
for i in range(num_of_sites):
    for j in range(0, num_of_attributes):
        data[i][j] = 0 if data[i][j] == 'n' else 1
saum = np.array(data, dtype=np.int32)

aaum = np.dot(saum.T, saum)

from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2)
distances = kmeans.fit_transform(saum)
attr_fragment = np.zeros(len(distances), dtype=np.int32)
for i in range(len(distances)):
    attr_fragment[i] = np.argmin(distances[i]) + 1
distance_frag = np.column_stack((distances, attr_fragment))

df = pd.DataFrame(distance_frag)

df1 = df.iloc[:, 0:-1]
df1.columns = ["F1", "F2"]

fragment_allocation_kmeans = pd.DataFrame(index=user_query_list.index)
for i in range(1, 3):
    fragment_indices_kmeans = np.where(attr_fragment == i)[0]
    attributes_assigned = np.zeros(num_of_attributes, dtype=int)
    attributes_assigned[fragment_indices_kmeans] = 1
    fragment_allocation_kmeans[f'AttrFromFragment{i}'] = np.where(attributes_assigned == 1, user_query_list.columns, None)

frag_all = pd.DataFrame(fragment_allocation_kmeans)
#print(frag_all)

        A1 A2 A3   A4 A5 A6 A7 A8   A9 A10
sites                                     
1        c  n  n    n  n  n  n  c    n   n
2        r  n  r    n  n  n  n  n    n   r
3        n  r  n    n  n  r  n  n    n   n
4        c  n  n    n  n  n  c  c    n   n
5        n  n  n  r,r  r  n  r  r    r   n
6        n  n  n    r  r  n  r  n    n   n
7        n  n  r    n  n  r  n  r    n   r
8        d  d  d    d  d  d  d  d  d,r   d
9      d,r  d  d    d  d  d  d  d    d   d
10       c  c  n    n  n  n  n  n    n   n


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
from skfuzzy.cluster import cmeans
ans= cmeans(saum,c=2,m=2,maxiter=200,error=0)
print(np.array(ans[1].T))

distances_fcmeans = np.array(ans[1]).T
attr_fragment_cmeans = np.zeros(len(distances_fcmeans),dtype=np.int32)
for i in range(len(distances_fcmeans)):
    attr_fragment_cmeans[i] = np.argmax(distances_fcmeans[i])+1
distance_frag_cmeans = np.column_stack((distances_fcmeans,attr_fragment_cmeans))
distance_frag_cmeans
df = pd.DataFrame(distance_frag_cmeans)

df2 =  df.iloc[:,0:-1]
df2.columns = ["F1", "F2"]
df2
fragment_allocation_cmeans=np.zeros((saum.shape[0],2),dtype=np.int32)
for i in range(len(saum)):
    total_access = 0
    for j in range(len(saum[0])):
        if saum[i][j] == 1 and distance_frag_cmeans[j][2]==1:
            total_access += saum[i][j]
    fragment_allocation_cmeans[i][0]=total_access
fragment_allocation_cmeans

for i in range(len(saum)):
    total_access = 0
    for j in range(len(saum[0])):
        if saum[i][j] == 1 and distance_frag_cmeans[j][2]==2:
            total_access += saum[i][j]
    fragment_allocation_cmeans[i][1]=total_access
fragment_allocation_cmeans = pd.DataFrame((fragment_allocation_cmeans))
fragment_allocation_cmeans.columns=['AttrFromFragmentOne','AttrFromFragmentTwo']
fragment_allocation_cmeans.index=user_query_list.index
fragment_allocation_cmeans

[[0.38460908 0.61539092]
 [0.40998672 0.59001328]
 [0.12911944 0.87088056]
 [0.91672001 0.08327999]
 [0.91672001 0.08327999]
 [0.26811203 0.73188797]
 [0.84215468 0.15784532]
 [0.51079376 0.48920624]
 [0.77329178 0.22670822]
 [0.12911944 0.87088056]]


,AttrFromFragmentOne,AttrFromFragmentTwo
sites,,
1,1,1
2,0,3
3,0,2
4,2,1
5,5,0
6,3,0
7,1,3
8,5,5
9,5,5


In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

user_query_list = pd.read_csv('/content/drive/MyDrive/ddbss.csv', index_col='sites')
user_query_list = user_query_list.fillna('n')
user_query_list = user_query_list.sort_values(by=['sites'])
print(user_query_list)
data = user_query_list.to_numpy()
num_of_attributes = data[0].shape[0]
num_of_sites = data.shape[0]
for i in range(num_of_sites):
    for j in range(0, num_of_attributes):
        data[i][j] = 0 if data[i][j] == 'n' else 1
saum = np.array(data, dtype=np.int32)


aaum = np.dot(saum.T, saum)

from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2)
distances = kmeans.fit_transform(saum)
attr_fragment = np.zeros(len(distances), dtype=np.int32)
for i in range(len(distances)):
    attr_fragment[i] = np.argmin(distances[i]) + 1
distance_frag = np.column_stack((distances, attr_fragment))

df = pd.DataFrame(distance_frag)

df1 = df.iloc[:, 0:-1]
df1.columns = ["F1", "F2"]

fragment_allocation=np.zeros((saum.shape[0],2),dtype=np.int32)
for i in range(len(saum)):
    total_access = 0
    for j in range(len(saum[0])):
        if saum[i][j] == 1 and distance_frag[j][2] == 1:
            total_access += saum[i][j]
    fragment_allocation[i][0] = total_access

for i in range(len(saum)):
    total_access = 0
    for j in range(len(saum[0])):
        if saum[i][j] == 1 and distance_frag[j][2] == 2:
            total_access += saum[i][j]
    fragment_allocation[i][1] = total_access

frag_all = pd.DataFrame(fragment_allocation)
frag_all.columns = ['AttrFromFragmentOne', 'AttrFromFragmentTwo']
frag_all.index = user_query_list.index

# Print the result
print(frag_all)

        A1 A2 A3   A4 A5 A6 A7 A8   A9 A10
sites                                     
1        c  n  n    n  n  n  n  c    n   n
2        r  n  r    n  n  n  n  n    n   r
3        n  r  n    n  n  r  n  n    n   n
4        c  n  n    n  n  n  c  c    n   n
5        n  n  n  r,r  r  n  r  r    r   n
6        n  n  n    r  r  n  r  n    n   n
7        n  n  r    n  n  r  n  r    n   r
8        d  d  d    d  d  d  d  d  d,r   d
9      d,r  d  d    d  d  d  d  d    d   d
10       c  c  n    n  n  n  n  n    n   n
       AttrFromFragmentOne  AttrFromFragmentTwo
sites                                          
1                        1                    1
2                        3                    0
3                        1                    1
4                        2                    1
5                        2                    3
6                        2                    1
7                        2                    2
8                        6                    4
9   

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

user_query_list = pd.read_csv('/content/drive/MyDrive/ddbss.csv', index_col='sites')
user_query_list = user_query_list.fillna('n')
user_query_list = user_query_list.sort_values(by=['sites'])
# print(user_query_list)

data = user_query_list.to_numpy()
num_of_attributes = data[0].shape[0]
num_of_sites = data.shape[0]
for i in range(num_of_sites):
    for j in range(0, num_of_attributes):
        data[i][j] = 0 if data[i][j] == 'n' else 1
saum = np.array(data, dtype=np.int32)

kmeans = KMeans(n_clusters=2)
distances_kmeans = kmeans.fit_transform(saum)
attr_fragment_kmeans = np.zeros(len(distances_kmeans), dtype=np.int32)
for i in range(len(distances_kmeans)):
    attr_fragment_kmeans[i] = np.argmin(distances_kmeans[i]) + 1

fragment_allocation_kmeans = pd.DataFrame(index=user_query_list.index)
for i in range(1, 3):
    fragment_indices_kmeans = np.where(attr_fragment_kmeans == i)[0]
    fragment_allocation_kmeans[f'AttrFromFragment{i}'] = 0
    fragment_allocation_kmeans.loc[user_query_list.index[fragment_indices_kmeans], f'AttrFromFragment{i}'] = 1



for i in range(1, 3):
    print(f'Attributes in Fragment {i}: {user_query_list.columns[fragment_allocation_kmeans[f"AttrFromFragment{i}"] == 1].tolist()}')

Attributes in Fragment 1: ['A5', 'A6', 'A8', 'A9']
Attributes in Fragment 2: ['A1', 'A2', 'A3', 'A4', 'A7', 'A10']


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

user_query_list = pd.read_csv('/content/drive/MyDrive/ddbss.csv', index_col='sites')
user_query_list = user_query_list.fillna('n')
user_query_list = user_query_list.sort_values(by=['sites'])


data = user_query_list.to_numpy()
num_of_attributes = data[0].shape[0]
num_of_sites = data.shape[0]
for i in range(num_of_sites):
    for j in range(0, num_of_attributes):
        data[i][j] = 0 if data[i][j] == 'n' else 1
saum = np.array(data, dtype=np.int32)

kmeans = KMeans(n_clusters=2)
distances_kmeans = kmeans.fit_transform(saum)
attr_fragment_kmeans = np.zeros(len(distances_kmeans), dtype=np.int32)
for i in range(len(distances_kmeans)):
    attr_fragment_kmeans[i] = np.argmin(distances_kmeans[i]) + 1

fragment_allocation_kmeans = pd.DataFrame(index=user_query_list.index)
for i in range(1, 3):
    fragment_indices_kmeans = np.where(attr_fragment_kmeans == i)[0]
    fragment_allocation_kmeans[f'AttrFromFragment{i}'] = 0
    fragment_allocation_kmeans.loc[user_query_list.index[fragment_indices_kmeans], f'AttrFromFragment{i}'] = 1


for i in range(1, 3):
    print(f'Attributes in Fragment {i}: {user_query_list.columns[fragment_allocation_kmeans[f"AttrFromFragment{i}"] == 1].tolist()}')

Attributes in Fragment 1: ['A5', 'A6', 'A8', 'A9']
Attributes in Fragment 2: ['A1', 'A2', 'A3', 'A4', 'A7', 'A10']


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [ ]:
ans = cmeans(saum, c=2, m=2, maxiter=200, error=0)
distances_fcmeans = np.array(ans[1]).T

attr_fragment_cmeans = np.zeros(len(distances_fcmeans), dtype=np.int32)
for i in range(len(distances_fcmeans)):
    attr_fragment_cmeans[i] = np.argmax(distances_fcmeans[i]) + 1

distance_frag_cmeans = np.column_stack((distances_fcmeans, attr_fragment_cmeans))
df = pd.DataFrame(distance_frag_cmeans)
df2 = df.iloc[:, 0:-1]
df2.columns = ["F1", "F2"]


fragment_allocation_cmeans = pd.DataFrame(index=user_query_list.index)
for i in range(1, 3):
    fragment_indices_cmeans = np.where(attr_fragment_cmeans == i)[0]
    attributes_assigned = user_query_list.columns[fragment_indices_cmeans].tolist()


    attributes_assigned.extend(['None'] * (len(user_query_list.columns) - len(attributes_assigned)))

    fragment_allocation_cmeans[f'AttrFromFragment{i}'] = attributes_assigned

for i in range(1, 3):
    attributes_in_fragment_i = fragment_allocation_cmeans[f'AttrFromFragment{i}'].tolist()
    print(f"Attributes in Fragment {i}: {attributes_in_fragment_i}")

Attributes in Fragment 1: ['A1', 'A2', 'A3', 'A6', 'A10', 'None', 'None', 'None', 'None', 'None']
Attributes in Fragment 2: ['A4', 'A5', 'A7', 'A8', 'A9', 'None', 'None', 'None', 'None', 'None']
